In [1]:
# Import Numpy & PyTorch
import numpy as np
import torch

## Linear Regression Model using PyTorch built-ins

Let's re-implement the same model using some built-in functions and classes from PyTorch.

And now using two different targets: Apples and Oranges

In [2]:
# Imports
import torch.nn as nn

In [3]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], [91, 88, 64], [87, 134, 58], [102, 43, 37], [69, 96, 70], [73, 67, 43], [91, 88, 64], [87, 134, 58], [102, 43, 37], [69, 96, 70], [73, 67, 43], [91, 88, 64], [87, 134, 58], [102, 43, 37], [69, 96, 70]], dtype='float32')
# Targets (apples, oranges)
targets = np.array([[56, 70], [81, 101], [119, 133], [22, 37], [103, 119], 
                    [56, 70], [81, 101], [119, 133], [22, 37], [103, 119], 
                    [56, 70], [81, 101], [119, 133], [22, 37], [103, 119]], dtype='float32')

In [4]:
inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

### Dataset and DataLoader

We'll create a `TensorDataset`, which allows access to rows from `inputs` and `targets` as tuples. We'll also create a DataLoader, to split the data into batches while training. It also provides other utilities like shuffling and sampling.

In [5]:
# Import tensor dataset & data loader
from torch.utils.data import TensorDataset, DataLoader

In [6]:
# Define dataset
#The TensorDataset allows us to access a small section of the training data using the array indexing notation ([0:3] in the above code). 
#It returns a tuple (or pair), in which the first element contains the input variables for the selected rows, and the second contains the targets.
train_ds = TensorDataset(inputs, targets)
train_ds[0:3]

(tensor([[ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.]]), tensor([[ 56.,  70.],
         [ 81., 101.],
         [119., 133.]]))

In [7]:
# Define data loader
#DataLoader, which can split the data into batches of a predefined size while training. 
#It also provides other utilities like shuffling and random sampling of the data
from torch.utils.data import DataLoader
batch_size = 5
train_dl = DataLoader(train_ds, batch_size, shuffle=True)

### nn.Linear
Instead of initializing the weights & biases manually, we can define the model using `nn.Linear`.

In [8]:
# Define model
model = nn.Linear(3, 2)
print(model.weight)
print(model.bias)

Parameter containing:
tensor([[-0.4583,  0.0757, -0.2839],
        [-0.4806,  0.0842, -0.4936]], requires_grad=True)
Parameter containing:
tensor([0.3717, 0.2886], requires_grad=True)


### Optimizer
Instead of manually manipulating the weights & biases using gradients, we can use the optimizer `optim.SGD`.

In [9]:
# Define optimizer
opt = torch.optim.SGD(model.parameters(), lr=1e-5)

### Loss Function
Instead of defining a loss function manually, we can use the built-in loss function `mse_loss`.

In [10]:
# Import nn.functional
import torch.nn.functional as F

In [11]:
# Define loss function
loss_fn = F.mse_loss

In [12]:
loss = loss_fn(model(inputs), targets)
print(loss)

tensor(20324.2500, grad_fn=<MseLossBackward>)


### Train the model

We are ready to train the model now. We can define a utility function `fit` which trains the model for a given number of epochs.

In [13]:
# Utility function to train the model
def fit(num_epochs, model, loss_fn, opt):
    
    # Repeat for given number of epochs
    for epoch in range(num_epochs):
        
        # Train with batches of data
        for xb,yb in train_dl:
            
            # 1. Generate predictions
            pred = model(xb)
            
            # 2. Calculate loss
            loss = loss_fn(pred, yb)
            
            # 3. Compute gradients
            loss.backward()
            
            # 4. Update parameters using gradients
            opt.step()
            
            # 5. Reset the gradients to zero
            opt.zero_grad()
        
        # Print the progress
        if (epoch+1) % 10 == 0:
            print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

In [14]:
# Train the model for 100 epochs
fit(100, model, loss_fn, opt)

Epoch [10/100], Loss: 262.3423
Epoch [20/100], Loss: 72.0730
Epoch [30/100], Loss: 152.7133
Epoch [40/100], Loss: 53.4528
Epoch [50/100], Loss: 47.9484
Epoch [60/100], Loss: 33.9337
Epoch [70/100], Loss: 59.7691
Epoch [80/100], Loss: 46.0382
Epoch [90/100], Loss: 23.5024
Epoch [100/100], Loss: 39.0346


In [15]:
# Generate predictions
preds = model(inputs)
preds

tensor([[ 58.2715,  71.5718],
        [ 79.6646,  96.1004],
        [123.0401, 141.9145],
        [ 26.5427,  43.0508],
        [ 94.3601, 107.6814],
        [ 58.2715,  71.5718],
        [ 79.6646,  96.1004],
        [123.0401, 141.9145],
        [ 26.5427,  43.0508],
        [ 94.3601, 107.6814],
        [ 58.2715,  71.5718],
        [ 79.6646,  96.1004],
        [123.0401, 141.9145],
        [ 26.5427,  43.0508],
        [ 94.3601, 107.6814]], grad_fn=<AddmmBackward>)

In [16]:
# Compare with targets
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])

Now we can define the model, optimizer and loss function exactly as before.

In [17]:
fit(200, model, loss_fn, opt)

Epoch [10/200], Loss: 41.6884
Epoch [20/200], Loss: 43.1515
Epoch [30/200], Loss: 23.6469
Epoch [40/200], Loss: 29.6282
Epoch [50/200], Loss: 27.9685
Epoch [60/200], Loss: 39.1126
Epoch [70/200], Loss: 24.3601
Epoch [80/200], Loss: 21.8778
Epoch [90/200], Loss: 27.0414
Epoch [100/200], Loss: 25.0338
Epoch [110/200], Loss: 19.7745
Epoch [120/200], Loss: 27.6139
Epoch [130/200], Loss: 17.7455
Epoch [140/200], Loss: 3.1093
Epoch [150/200], Loss: 15.5048
Epoch [160/200], Loss: 14.8312
Epoch [170/200], Loss: 9.2048
Epoch [180/200], Loss: 12.7000
Epoch [190/200], Loss: 12.4498
Epoch [200/200], Loss: 7.5158


#Exercise 1:
 Try Linear Regression just using numpy (Without Tensorflow/Pytorch or other torch library). You can optionally use sklearn (if you want)
#Exercise 2:
 Try Linear regression on same prediction data using Tensorflow

 